In [12]:
# For macOS/Linux
ollama serve  # Keep this running in a separate terminal

# For Windows
ollama.exe serve

SyntaxError: invalid syntax (2982790699.py, line 2)

In [ ]:
# In Jupyter notebook
from evaluation.qualitative import human_evaluation
from src.utils import load_briefs

briefs = load_briefs("data/briefs.json")
matcher = ProjectMatcher()
human_evaluation(matcher, briefs[:3])  # Test first 3 projects

In [10]:
import sys
import platform
from pathlib import Path
from typing import List
import json
import re

# Platform-specific optimizations
if platform.system() == "Windows":
    from ctypes import windll
    kernel32 = windll.kernel32
    kernel32.SetConsoleMode(kernel32.GetStdHandle(-11), 7)

class CrossPlatformProjectMatcher:
    def __init__(self):
        # Detect OS for model selection
        self._init_embedding_model()
        self._init_llm()
        self.collection = self._init_chroma()

    def _init_embedding_model(self):
        # Use different models based on hardware
        if platform.machine() in ['arm64', 'aarch64']:
            self.embed_model = SentenceTransformer('all-MiniLM-L6-v2')
        else:
            self.embed_model = SentenceTransformer('paraphrase-albert-small-v2')

    def _init_llm(self):
        # Select model based on available RAM
        try:
            self.llm = Ollama(model="llama3")
        except:
            self.llm = Ollama(model="phi3:mini")  # Fallback for low RAM

    def _init_chroma(self):
        # OS-agnostic path handling
        chroma_path = Path.home() / ".cache" / "chroma"
        return chromadb.PersistentClient(str(chroma_path))
    
    def get_ollama_endpoint(self):
        # Windows/MacOS/Linux compatible endpoint
        if platform.system() == "Windows":
            return "http://localhost:11434"
        else:
            return "unix:///var/run/ollama/ollama.sock"

In [7]:
from sentence_transformers import SentenceTransformer

# Replace OpenAI embeddings with local model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text: str) -> list[float]:
    return embedding_model.encode(text).tolist()

In [8]:
from src.models import ProjectBrief, Expert
from src.chroma_setup import get_chroma_collection
import os
import json
import re
from typing import List
from langchain_community.llms import Ollama
from sentence_transformers import SentenceTransformer
from tenacity import retry, stop_after_attempt, wait_exponential

class LocalProjectMatcher:
    def __init__(self):
        # Local embedding model
        self.embed_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Local LLM
        self.llm = Ollama(
            model="llama3",
            temperature=0.3,
            num_ctx=4096  # Context window size
        )
        
        # ChromaDB collection
        self.collection = get_chroma_collection()
    
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
    def find_matches(self, brief: ProjectBrief, top_k: int = 5) -> List[Expert]:
        # Generate local embedding
        query_embedding = self.embed_model.encode(brief.description).tolist()
        
        # Query ChromaDB
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k*2,
            where={"domain": brief.domain},
            include=["metadatas", "documents"]
        )
        
        # Process candidates
        candidates = [
            Expert(**meta) 
            for meta in results['metadatas'][0]
            if any(skill in meta['skills'] for skill in brief.required_skills)
        ]
        
        return self._process_llm_response(brief, candidates)[:top_k]
    
    def _process_llm_response(self, brief: ProjectBrief, candidates: List[Expert]):
        prompt = f"""SYSTEM: You're an expert matchmaker between projects and professionals.
        PROJECT: {brief.title}
        DESCRIPTION: {brief.description[:500]}
        REQUIRED SKILLS: {', '.join(brief.required_skills)}
        
        CANDIDATES:
        {chr(10).join(f"- {e.name}: {', '.join(e.skills)}" for e in candidates)}
        
        Return JSON array with top matches sorted by relevance:
        {{"matches": [{{"name": "expert_name", "score": 0-10}}]}}
        """
        
        try:
            response = self.llm.invoke(prompt)
            return self._parse_response(response)
        except json.JSONDecodeError:
            return self._fallback_ranking(candidates)
    
    def _parse_response(self, response: str) -> List[Expert]:
        # Try to extract JSON from response
        try:
            json_str = re.search(r'\{.*\}', response, re.DOTALL).group()
            data = json.loads(json_str)
            return sorted(data['matches'], key=lambda x: x['score'], reverse=True)
        except:
            return self._fallback_ranking(response)
    
    def _fallback_ranking(self, candidates):
        # Simple alphabetical fallback
        return sorted(candidates, key=lambda x: x.name)

In [11]:
# Initialize
matcher = LocalProjectMatcher()

# Sample brief
medical_brief = ProjectBrief(
    title="Medical Chatbot",
    description="Develop an AI chatbot for patient triage in hospitals",
    required_skills=["NLP", "Healthcare"],
    domain="Healthcare"
)

# Get matches
matches = matcher.find_matches(medical_brief)
print("Top Matches:")
for expert in matches:
    print(f"- {expert.name} ({expert.skills})")

RetryError: RetryError[<Future at 0x30f90b450 state=finished raised ConnectionError>]